In [442]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as const

#modern earth albedo file
absor1 = pd.read_csv('/Users/huihaoz/Downloads/GeometricA_Earth_HighCloud_UltraRes1.dat', comment = '#', delimiter='\s+')

#suitable planets
pts=pd.read_csv('/Users/huihaoz/Downloads/PT_PES.txt', sep=',')

#define the tempuratre of them
pts["Teq"] = (pts.STK * ((pts.SRadiusSun *  const.R_sun/(2*pts.SMAAU * const.au)))**(1/2) * (1-0.3)**(0.25))
pts.iloc[0,8]=181.7
pts.iloc[3,8]=351
pts.iloc[9,8]=468
pts.iloc[11,8]=301

# Normalization of albedo
# Split a Python List into Chunks using For Loops
our_list = np.array(absor1.albH2O[:1079300])
chunked_list = list()
chunk_size = 251*2*5
for i in range(0, len(our_list), chunk_size):
    chunked_list.append(our_list[i:i+chunk_size])
    
our_listwl = np.array(absor1.nm[:1079300] / 1000)
chunked_listwl = list()
chunk_sizewl = 251*2*5
for i in range(0, len(our_listwl), chunk_sizewl):
    chunked_listwl.append(our_listwl[i:i+chunk_sizewl])
    
y_fit_index = np.linspace(0,429,430,dtype = int)
wl_fit_index = [np.argmax(chunked_list[i]) for i in y_fit_index] #just for check

pn_fit_non = [max(chunked_list[i]) for i in y_fit_index]
wl_fit_non = [chunked_listwl[i][np.argmax(chunked_list[i])] for i in y_fit_index]
alb_nogas = np.interp(absor1.nm/1000,wl_fit_non,np.array(pn_fit_non))
wl_um2 = np.array(absor1.nm[:1079300])/1000 * u.um
my_list_norabd = (absor1.totalb / alb_nogas) #normalized abd

# Iterate over the list and replace any value greater than 1 with 1
for i in range(len(my_list_norabd)):
    if my_list_norabd[i] > 1:
        my_list_norabd[i] = 1
tbd_planet=[]

#balckbody for target planets with normalzed abd
for i in range(len(pts.Teq)):
    tbd_planet.append(np.array(my_list_norabd[:1079300]) * 2 * const.h *const.c**2 / ((wl_um2)**5 * (np.e**(const.c * const.h / (wl_um2 *const.k_B * pts.Teq[i] * u.K)) - 1)))

#distance
dis_2 = pts.DistancePC* const.pc

#rad
d_s_2 = pts.RadiusEar * const.R_earth

#solid angle =  pi * r^2 / D^2
ste = (d_s_2**2 * np.pi / dis_2**2)
tbd_planet_earth = []
for i in range(len(tbd_planet)):

    tbd_planet_earth.append((tbd_planet[i] * ste[i]).to(u.erg/(u.s * u.cm**2 * u.um)))
Flux_tbd_planet_earth =[]
for i in range(len(tbd_planet_earth)):
    Flux_tbd_planet_earth.append((tbd_planet_earth[i][1:] * np.diff(wl_um2)).to(u.W/u.m**2))

#load the star flux model
f_tra0 = pd.read_csv('/Users/huihaoz/Downloads/lte030.0-5.0-0.0a+0.0.BT-Settl.spec.7.txt', comment = '#', delimiter='\s+')
f_tra1 = pd.read_csv('/Users/huihaoz/Downloads/lte030.0-5.0-0.0a+0.0.BT-Settl.spec.7.txt', comment = '#', delimiter='\s+')
f_tra2 = pd.read_csv('/Users/huihaoz/Downloads/lte036.0-5.0-0.0a+0.0.BT-Settl.spec.7.txt', comment = '#', delimiter='\s+')
f_tra3 = pd.read_csv('/Users/huihaoz/Downloads/lte035.0-5.0-0.0a+0.0.BT-Settl.spec.7.txt', comment = '#', delimiter='\s+')
f_tra4 = pd.read_csv('/Users/huihaoz/Downloads/lte034.0-5.0-0.0a+0.0.BT-Settl.spec.7.txt', comment = '#', delimiter='\s+')
f_tra5 = pd.read_csv('/Users/huihaoz/Downloads/lte037.0-5.0-0.0a+0.0.BT-Settl.spec.7.txt', comment = '#', delimiter='\s+')
f_tra6 = pd.read_csv('/Users/huihaoz/Downloads/lte035.0-5.0-0.0a+0.0.BT-Settl.spec.7.txt', comment = '#', delimiter='\s+')
f_tra7 = pd.read_csv('/Users/huihaoz/Downloads/lte034.0-4.5-0.0a+0.0.BT-Settl.spec.7.txt', comment = '#', delimiter='\s+')
f_tra8 = pd.read_csv('/Users/huihaoz/Downloads/lte030.0-5.0-0.0a+0.0.BT-Settl.spec.7.txt', comment = '#', delimiter='\s+')
f_tra9 = pd.read_csv('/Users/huihaoz/Downloads/lte037.0-5.0-0.0a+0.0.BT-Settl.spec.7.txt', comment = '#', delimiter='\s+')
f_tra10 = pd.read_csv('/Users/huihaoz/Downloads/lte029.0-5.0-0.0a+0.0.BT-Settl.spec.7.txt', comment = '#', delimiter='\s+')
f_tra11 = pd.read_csv('/Users/huihaoz/Downloads/lte032.0-5.0-0.0a+0.0.BT-Settl.spec.7.txt', comment = '#', delimiter='\s+')
f_tra12 = pd.read_csv('/Users/huihaoz/Downloads/lte029.0-5.5-0.0a+0.0.BT-Settl.spec.7.txt', comment = '#', delimiter='\s+')
f_tra13 = pd.read_csv('/Users/huihaoz/Downloads/lte033.0-5.0-0.0a+0.0.BT-Settl.spec.7.txt', comment = '#', delimiter='\s+')
f_tra14 = pd.read_csv('/Users/huihaoz/Downloads/lte026.0-5.0-0.0a+0.0.BT-Settl.spec.7.txt', comment = '#', delimiter='\s+')
f_tra15 = pd.read_csv('/Users/huihaoz/Downloads/lte026.0-5.0-0.0a+0.0.BT-Settl.spec.7.txt', comment = '#', delimiter='\s+')
f_traErgsseccmA=np.array([np.array(f_tra0.ErgsseccmA),np.array(f_tra1.ErgsseccmA),np.array(f_tra2.ErgsseccmA),np.array(f_tra3.ErgsseccmA),np.array(f_tra4.ErgsseccmA),np.array(f_tra5.ErgsseccmA),np.array(f_tra6.ErgsseccmA),np.array(f_tra7.ErgsseccmA),np.array(f_tra8.ErgsseccmA),np.array(f_tra9.ErgsseccmA),np.array(f_tra10.ErgsseccmA),np.array(f_tra11.ErgsseccmA),np.array(f_tra12.ErgsseccmA),np.array(f_tra13.ErgsseccmA),np.array(f_tra14.ErgsseccmA),np.array(f_tra15.ErgsseccmA)])
f_tra1_fum0=[10**(i-8) * u.erg/(u.s * u.cm**2 *u.Angstrom) for i in f_traErgsseccmA]
f_traAng=np.array([np.array(f_tra0.Angstroem / 10000),np.array(f_tra1.Angstroem / 10000),np.array(f_tra2.Angstroem / 10000),np.array(f_tra3.Angstroem / 10000),np.array(f_tra4.Angstroem / 10000),np.array(f_tra5.Angstroem / 10000),np.array(f_tra6.Angstroem / 10000),np.array(f_tra7.Angstroem / 10000),np.array(f_tra8.Angstroem / 10000),np.array(f_tra9.Angstroem / 10000),np.array(f_tra10.Angstroem / 10000),np.array(f_tra11.Angstroem / 10000),np.array(f_tra12.Angstroem / 10000),np.array(f_tra13.Angstroem / 10000),np.array(f_tra14.Angstroem / 10000),np.array(f_tra15.Angstroem / 10000)])


/var/folders/6t/cpjtlkn10wv603vnph9nr3sh0000gn/T/ipykernel_22589/2583433914.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  f_traErgsseccmA=np.array([np.array(f_tra0.ErgsseccmA),np.array(f_tra1.ErgsseccmA),np.array(f_tra2.ErgsseccmA),np.array(f_tra3.ErgsseccmA),np.array(f_tra4.ErgsseccmA),np.array(f_tra5.ErgsseccmA),np.array(f_tra6.ErgsseccmA),np.array(f_tra7.ErgsseccmA),np.array(f_tra8.ErgsseccmA),np.array(f_tra9.ErgsseccmA),np.array(f_tra10.ErgsseccmA),np.array(f_tra11.ErgsseccmA),np.array(f_tra12.ErgsseccmA),np.array(f_tra13.ErgsseccmA),np.array(f_tra14.ErgsseccmA),np.array(f_tra15.ErgsseccmA)])
/var/folders/6t/cpjtlkn10wv603vnph9nr3sh0000gn/T/ipykernel_22589/2583433914.py:74: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which i

In [445]:
#interp the star flux based on the wavelength of albedo
f_ref_earth_fit =[]
for i in range(len(f_tra1_fum0)):
    f_ref_earth_fit.append(np.interp(absor1.nm[:1079300]/1000,f_traAng[i],f_tra1_fum0[i]))

In [446]:
#convert the unit from J/ s cm^2 ans to W / m^2
Flux_surface = []
for i in range(len(f_ref_earth_fit)):
    Flux_surface.append((f_ref_earth_fit[i][1:] * np.diff(absor1.nm[:1079300]/1000) * u.um).to(u.W/u.m**2))

In [447]:
#delution factor
df = (pts.SRadiusSun * const.R_sun / (pts.SMAAU * const.au))**2

#flux of star on planet
f_ref_p = np.array(Flux_surface)* np.array(df).reshape(-1,1)

#flux of planet on earth, without abd
f_ref_earth = f_ref_p* np.pi * np.array(((pts.RadiusEar * const.R_earth)**2 / (4 * np.pi * (pts.DistancePC * const.pc)**2))).reshape(-1,1)
f_ref_earth_fit = f_ref_earth 
modern_earth_abd_fit = np.array(my_list_norabd[1:1079300])*0.3
f_ref_planet_earth_fit = [i * modern_earth_abd_fit for i in f_ref_earth_fit] #w/m2

#total flux of planet on earth
flux_tot = np.array(f_ref_planet_earth_fit)+np.array(Flux_tbd_planet_earth)

#target band
wl_um_check = np.linspace(0.5,2.45,1950) * u.um


#total flux after bin
Range_min=wl_um_check-np.diff(wl_um_check)[1]*0.5
Range_max=wl_um_check+np.diff(wl_um_check)[1]*0.5
bin_range=[]
for i in range(len(wl_um_check)):
    bin_range.append(np.where((absor1.nm[1:1079300]/1000 < Range_max[i]/u.um) & (absor1.nm[1:1079300]/1000 > Range_min[i]/u.um)))
flux_bin = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]#W/m**2
for a in range(len(flux_tot)):
    for i in range(len(bin_range)):
        flux_bin[a].append(np.sum(flux_tot[a][bin_range[i]]))
        

# NO GAS flux
#blackbody
my_list_Noabd = alb_nogas*0+1
tbd_planet_no=[]

for i in range(len(pts.Teq)):
    tbd_planet_no.append(np.array(my_list_Noabd[:1079300]) * 2 * const.h *const.c**2 / ((wl_um2)**5 * (np.e**(const.c * const.h / (wl_um2 *const.k_B * pts.Teq[i] * u.K)) - 1)))
tbd_planet_earth_no = []
for i in range(len(tbd_planet_no)):
    tbd_planet_earth_no.append((tbd_planet_no[i] * ste[i]).to(u.erg/(u.s * u.cm**2 * u.um)))
Flux_tbd_planet_earth_no =[]
for i in range(len(tbd_planet_earth_no)):
    Flux_tbd_planet_earth_no.append((tbd_planet_earth_no[i][1:] * np.diff(wl_um2)).to(u.W/u.m**2))
##
#reflection flux
modern_earth_abd_fit_no = np.array(my_list_norabd[1:1079300])*0 + 0.3
f_ref_planet_earth_fit_no = [i * modern_earth_abd_fit_no for i in f_ref_earth_fit]

#total flux and flux bin
flux_tot_no = np.array(f_ref_planet_earth_fit_no)+np.array(Flux_tbd_planet_earth_no)

flux_bin_no = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]#W/m**2
for a in range(len(flux_tot_no)):
    for i in range(len(bin_range)):
        flux_bin_no[a].append(np.sum(flux_tot_no[a][bin_range[i]]))

---

# Star

In [448]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as const
from astropy.nddata import NDData

In [449]:
# energy of photon
h = const.h
wl_um = np.linspace(0.5,2.45,1950) * u.um
c = const.c #unit:m/s
wl_m = (wl_um).to(u.m)
pe_1 = h * c / (wl_um).to(u.um)
wl_um_check = np.linspace(0.5,2.45,1950)*u.um

In [450]:
#delution factor of star on earth
df_s = (pts.SRadiusSun * const.R_sun / (pts.DistancePC * const.pc))**2

# flux of star on earth and bin
f_s = np.array(Flux_surface)*np.array(df_s).reshape(-1,1)
f_s_fit = f_s 
flux_bin_s = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]#W/m**2
for a in range(len(f_s_fit)):
    for i in range(len(bin_range)):
        flux_bin_s[a].append(np.sum(f_s_fit[a][bin_range[i]]))#more resolution, more data points, less photon number

In [1174]:
pts=pd.read_csv('/Users/huihaoz/Downloads/PT_PES.txt', sep=',')
R = 3500 # here we set Reseolution = 1000
t = 57960*u.s # here we set time = 1h with unit:s
S = (3930 * 0.5)**2 * np.pi *u.cm * u.cm #here we set diameter as the first diameter with unit:cm

In [1177]:
#photon number of star on ELT
pn_0_elt=(np.array(flux_bin_s[0])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_1_elt=(np.array(flux_bin_s[1])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_2_elt=(np.array(flux_bin_s[2])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_3_elt=(np.array(flux_bin_s[3])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_4_elt=(np.array(flux_bin_s[4])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_5_elt=(np.array(flux_bin_s[5])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_6_elt=(np.array(flux_bin_s[6])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_7_elt=(np.array(flux_bin_s[7])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_8_elt=(np.array(flux_bin_s[8])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_9_elt=(np.array(flux_bin_s[9])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_10_elt=(np.array(flux_bin_s[10])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_11_elt=(np.array(flux_bin_s[11])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_12_elt=(np.array(flux_bin_s[12])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_13_elt=(np.array(flux_bin_s[13])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_14_elt=(np.array(flux_bin_s[14])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_15_elt=(np.array(flux_bin_s[15])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)

In [1178]:
phe_1e=[i * u.W/u.m**2 for i in flux_bin] #energy of photon

In [1179]:
#photon number of planet on ELT
pn_0e_elt=(np.array(phe_1e[0])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_1e_elt=(np.array(phe_1e[1])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_2e_elt=(np.array(phe_1e[2])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_3e_elt=(np.array(phe_1e[3])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_4e_elt=(np.array(phe_1e[4])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_5e_elt=(np.array(phe_1e[5])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_6e_elt=(np.array(phe_1e[6])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_7e_elt=(np.array(phe_1e[7])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_8e_elt=(np.array(phe_1e[8])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_9e_elt=(np.array(phe_1e[9])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_10e_elt=(np.array(phe_1e[10])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_11e_elt=(np.array(phe_1e[11])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_12e_elt=(np.array(phe_1e[12])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_13e_elt=(np.array(phe_1e[13])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_14e_elt=(np.array(phe_1e[14])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_15e_elt=(np.array(phe_1e[15])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)

# Noise

In [1180]:
#sky
pe_1 = h * c / (wl_um).to(u.um) #energy of photon

# solid angle for ELT, working at 2um
ste_atmo2 = ((2*u.um / (39.3*u.m))**2 * u.steradian).to(u.arcsec**2) #ste

#sky emission
n_s = pd.read_csv('/Users/huihaoz/Documents/reseatch/radiance.dat.txt', comment = '#', delimiter='\s+')
wl_ns = (np.array(n_s["wavelength(nm)"])*u.nm).to(u.um)
bri_surface = np.array(n_s["ph/sec/arcsec^2/um/m^2"])*u.ph/(u.m**2 * u.s * u.arcsec**2 *u.um)
pe_ns = (h * c /wl_ns).to(u.J)/u.ph
st = ste_atmo2

#sky flux on earth
f_n_s = ((bri_surface *pe_ns).to(u.W/(u.m**2 * u.um * u.steradian))*st*np.diff(wl_ns)[1]).to(u.W/u.m**2)
bin_range_n = []

for i in range(len(wl_um)):
    bin_range_n.append(np.where((wl_ns/u.um < Range_max[i]/u.um) & (wl_ns/u.um > Range_min[i]/u.um)))

#flux of sky after bin
f_r_n=[]
for i in range(len(bin_range_n)):
    f_r_n.append(np.sum(f_n_s[bin_range_n[i]]).value)
pe_e = (h * c / wl_um).to(u.J)
noif_sky =(np.array(f_r_n)*u.W/u.m**2)

#photon number of sky on ELT
pnsky_1_elt=(noif_sky*S*t*wl_um/(pe_1*u.um)).to(u.m/u.m)

In [1181]:
#sky transmission
trans = pd.read_csv('/Users/huihaoz/Documents/reseatch/transmission.dat.txt', comment = '#', delimiter='\s+')

#interp with the wavelength we used
trans_use = np.interp(wl_um/u.um,trans["wavelength(nm)"]/1000,trans["transmission"])

# photon of star and planet encorded by Earth' atmosphere
pn_pla_elt=[pn_0e_elt,pn_1e_elt,pn_2e_elt,pn_3e_elt,pn_4e_elt,pn_5e_elt,pn_6e_elt,pn_7e_elt,pn_8e_elt,pn_9e_elt,pn_10e_elt,pn_11e_elt,pn_12e_elt,pn_13e_elt,pn_14e_elt,pn_15e_elt]
pn_star_elt=[pn_0_elt,pn_1_elt,pn_2_elt,pn_3_elt,pn_4_elt,pn_5_elt,pn_6_elt,pn_7_elt,pn_8_elt,pn_9_elt,pn_10_elt,pn_11_elt,pn_12_elt,pn_13_elt,pn_14_elt,pn_15_elt]

#sky transmission normalized
nor_trans = np.random.normal(trans_use,0.001*trans_use)

In [1182]:
#noise, includ of dectector(sky, planet, star) and the remove section and transmission
c_1 = 10**(-4)
n_0 = ((((pn_0e_elt+2*c_1*pn_0_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.01)**2 * pn_0e_elt**2)**(1/2)
n_1 = ((((pn_1e_elt+2*c_1*pn_1_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.01)**2 * pn_1e_elt**2)**(1/2)
n_2 = ((((pn_2e_elt+2*c_1*pn_2_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.01)**2 * pn_2e_elt**2)**(1/2)
n_3 = ((((pn_3e_elt+2*c_1*pn_3_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.01)**2 * pn_3e_elt**2)**(1/2)
n_4 = ((((pn_4e_elt+2*c_1*pn_4_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.01)**2 * pn_4e_elt**2)**(1/2)
n_5 = ((((pn_5e_elt+2*c_1*pn_5_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.01)**2 * pn_5e_elt**2)**(1/2)
n_6 = ((((pn_6e_elt+2*c_1*pn_6_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.01)**2 * pn_6e_elt**2)**(1/2)
n_7 = ((((pn_7e_elt+2*c_1*pn_7_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.01)**2 * pn_7e_elt**2)**(1/2)
n_8 = ((((pn_8e_elt+2*c_1*pn_8_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.01)**2 * pn_8e_elt**2)**(1/2)
n_9 = ((((pn_9e_elt+2*c_1*pn_9_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.01)**2 * pn_9e_elt**2)**(1/2)
n_10 = ((((pn_10e_elt+2*c_1*pn_10_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.01)**2 * pn_10e_elt**2)**(1/2)
n_11 = ((((pn_11e_elt+2*c_1*pn_11_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.01)**2 * pn_11e_elt**2)**(1/2)
n_12 = ((((pn_12e_elt+2*c_1*pn_12_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.01)**2 * pn_12e_elt**2)**(1/2)
n_13 = ((((pn_13e_elt+2*c_1*pn_13_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.01)**2 * pn_13e_elt**2)**(1/2)
n_14 = ((((pn_14e_elt+2*c_1*pn_14_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.01)**2 * pn_14e_elt**2)**(1/2)
n_15 = ((((pn_15e_elt+2*c_1*pn_15_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.01)**2 * pn_15e_elt**2)**(1/2)

# No gas

In [1183]:
#same as above but with albedo of no biosignature gas
phe_1e_nogas=[i * u.W/u.m**2 for i in flux_bin_no] #changed

In [1184]:
pn_0e_elt_nogas=(np.array(phe_1e_nogas[0])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_1e_elt_nogas=(np.array(phe_1e_nogas[1])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_2e_elt_nogas=(np.array(phe_1e_nogas[2])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_3e_elt_nogas=(np.array(phe_1e_nogas[3])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_4e_elt_nogas=(np.array(phe_1e_nogas[4])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_5e_elt_nogas=(np.array(phe_1e_nogas[5])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_6e_elt_nogas=(np.array(phe_1e_nogas[6])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_7e_elt_nogas=(np.array(phe_1e_nogas[7])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_8e_elt_nogas=(np.array(phe_1e_nogas[8])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_9e_elt_nogas=(np.array(phe_1e_nogas[9])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_10e_elt_nogas=(np.array(phe_1e_nogas[10])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_11e_elt_nogas=(np.array(phe_1e_nogas[11])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_12e_elt_nogas=(np.array(phe_1e_nogas[12])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_13e_elt_nogas=(np.array(phe_1e_nogas[13])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_14e_elt_nogas=(np.array(phe_1e_nogas[14])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)
pn_15e_elt_nogas=(np.array(phe_1e_nogas[15])*u.W/u.m**2 *S*t/(pe_1)).to(u.m/u.m)

In [1185]:
dn = np.array([pn_0e_elt_nogas-pn_0e_elt,pn_1e_elt_nogas-pn_1e_elt,pn_2e_elt_nogas-pn_2e_elt,pn_3e_elt_nogas-pn_3e_elt,pn_4e_elt_nogas-pn_4e_elt,pn_5e_elt_nogas-pn_5e_elt,pn_6e_elt_nogas-pn_6e_elt,pn_7e_elt_nogas-pn_7e_elt,pn_8e_elt_nogas-pn_8e_elt,pn_9e_elt_nogas-pn_9e_elt,pn_10e_elt_nogas-pn_10e_elt,pn_11e_elt_nogas-pn_11e_elt,pn_12e_elt_nogas-pn_12e_elt,pn_13e_elt_nogas-pn_13e_elt,pn_14e_elt_nogas-pn_14e_elt,pn_15e_elt_nogas-pn_15e_elt])
dnr = np.array([n_0,n_1,n_2,n_3,n_4,n_5,n_6,n_7,n_8,n_9,n_10,n_11,n_12,n_13,n_14,n_15])

# OK, all data prepared(above)
---

In [1186]:
#find the feature wavelength
h2o_f=(0.3-0.3*(absor1.albH2O[:1079300]/alb_nogas[:1079300]))
co2_f=(0.3-0.3*(absor1.albco2[:1079300]/alb_nogas[:1079300]))
ch4_f=(0.3-0.3*(absor1.albch4[:1079300]/alb_nogas[:1079300]))
o2_f=(0.3-0.3*(absor1.albo2[:1079300]/alb_nogas[:1079300]))

O2

In [1187]:
# Split a Python List into Chunks using For Loops
# find the envolop albedo (no o2)
our_list = np.array(o2_f)
chunked_list = list()
chunk_size = 251*5*2
for i in range(0, len(our_list), chunk_size):
    chunked_list.append(our_list[i:i+chunk_size])
    
our_listwl = np.array(absor1.nm[:1079300] / 1000)
chunked_listwl = list()
chunk_sizewl = 251*5*2
for i in range(0, len(our_listwl), chunk_sizewl):
    chunked_listwl.append(our_listwl[i:i+chunk_sizewl])
    
y_fit_index = np.linspace(0,429,430,dtype = int)
wl_fit_index = [np.argmax(chunked_list[i]) for i in y_fit_index] #just for check

pn_fit_non = [max(chunked_list[i]) for i in y_fit_index]
wl_fit_non = [chunked_listwl[i][np.argmax(chunked_list[i])] for i in y_fit_index]
interp_o2 = np.interp(wl_um/u.um,wl_fit_non,np.array(pn_fit_non))
baseline_feat = absor1.albo2[300000]*0.5*0.5
ind_o2 = np.where((interp_o2>0.15))

In [1188]:
# Split a Python List into Chunks using For Loops
# find the envolop albedo (no h2o)
our_list = np.array(h2o_f)
chunked_list = list()
chunk_size = 251*5*2
for i in range(0, len(our_list), chunk_size):
    chunked_list.append(our_list[i:i+chunk_size])
    
our_listwl = np.array(absor1.nm[:1079300] / 1000)
chunked_listwl = list()
chunk_sizewl = 251*5*2
for i in range(0, len(our_listwl), chunk_sizewl):
    chunked_listwl.append(our_listwl[i:i+chunk_sizewl])
    
y_fit_index = np.linspace(0,429,430,dtype = int)
wl_fit_index = [np.argmax(chunked_list[i]) for i in y_fit_index] #just for check

pn_fit_non = [max(chunked_list[i]) for i in y_fit_index]
wl_fit_non = [chunked_listwl[i][np.argmax(chunked_list[i])] for i in y_fit_index]
interp_h2o = np.interp(wl_um/u.um,wl_fit_non,np.array(pn_fit_non))
ind_h2o = np.where((interp_h2o>0.15))

In [1192]:
# Split a Python List into Chunks using For Loops
# find the envolop albedo (no ch4)
our_list = np.array(ch4_f)
chunked_list = list()
chunk_size = 251*5*2
for i in range(0, len(our_list), chunk_size):
    chunked_list.append(our_list[i:i+chunk_size])
    
our_listwl = np.array(absor1.nm[:1079300] / 1000)
chunked_listwl = list()
chunk_sizewl = 251*5*2
for i in range(0, len(our_listwl), chunk_sizewl):
    chunked_listwl.append(our_listwl[i:i+chunk_sizewl])
    
y_fit_index = np.linspace(0,429,430,dtype = int)
wl_fit_index = [np.argmax(chunked_list[i]) for i in y_fit_index] #just for check

pn_fit_non = [max(chunked_list[i]) for i in y_fit_index]
wl_fit_non = [chunked_listwl[i][np.argmax(chunked_list[i])] for i in y_fit_index]
interp_ch4 = np.interp(wl_um/u.um,wl_fit_non,np.array(pn_fit_non))
ind_ch4 = np.where((interp_ch4>0.15))

In [1193]:
# Split a Python List into Chunks using For Loops
# find the envolop albedo (no co2)
our_list = np.array(co2_f)
chunked_list = list()
chunk_size = 251*5*2
for i in range(0, len(our_list), chunk_size):
    chunked_list.append(our_list[i:i+chunk_size])
    
our_listwl = np.array(absor1.nm[:1079300] / 1000)
chunked_listwl = list()
chunk_sizewl = 251*5*2
for i in range(0, len(our_listwl), chunk_sizewl):
    chunked_listwl.append(our_listwl[i:i+chunk_sizewl])
    
y_fit_index = np.linspace(0,429,430,dtype = int)
wl_fit_index = [np.argmax(chunked_list[i]) for i in y_fit_index] #just for check

pn_fit_non = [max(chunked_list[i]) for i in y_fit_index]
wl_fit_non = [chunked_listwl[i][np.argmax(chunked_list[i])] for i in y_fit_index]
interp_co2 = np.interp(wl_um/u.um,wl_fit_non,np.array(pn_fit_non))
ind_co2 = np.where((interp_co2>0.15))

In [1194]:
#find wavelength range of them, remove the interval of overleap
lisco2 = ind_co2[0].tolist()
lish2o = ind_h2o[0].tolist()
liso2 = ind_o2[0].tolist()
lisch4 = ind_ch4[0].tolist()
set1 = set(lisco2)
set2 = set(lish2o)
set3 = set(liso2)
set4 = set(lisch4)
common = set1 & set2 | set1 & set3 | set1 & set4 | set2 & set3 | set2 & set4 | set3 & set4

In [1195]:
set1 -= common
set2 -= common
set3 -= common
set4 -= common

indco2 = list(set1)
indh2o = list(set2)
indo2 = list(set3)
indch4 = list(set4)

In [1196]:
indco2.sort()
indh2o.sort()
indo2.sort()
indch4.sort()

In [1202]:
#the total feature wavelength of atmosphere
indtotno = indco2+ indh2o + indo2 + indch4

In [1203]:
#find snr of o2
snr_o2_total=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
for a in range(16):
    snr_o2_total[a].append(np.sum((dn[a][indo2]/dnr[a][indo2])**2)**(1/2))

In [1204]:
snr_o2_total

[[0.6153105266861661],
 [1.9251161556904133],
 [8.992638484496645],
 [5.377232957113045],
 [2.062170746781315],
 [9.629131193468872],
 [2.884016878775699],
 [1.3988139308908536],
 [3.5965211749292125],
 [21.073747537332384],
 [4.097928121278342],
 [3.5205222622391217],
 [1.3022096734630264],
 [3.545087046707599],
 [0.5623563217519068],
 [0.08462259095250482]]

H2O

In [1205]:
#find snr of h2o
snr_h2o_total=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
for a in range(16):
    snr_h2o_total[a].append(np.sum((dn[a][indh2o]/dnr[a][indh2o])**2)**(1/2))

In [1206]:
snr_h2o_total

[[1.3658936319886594],
 [4.276357230878986],
 [16.01078322195538],
 [9.87015845512777],
 [3.904280838097297],
 [16.61371982571075],
 [5.292657221688262],
 [2.6950812379379183],
 [7.991405491655405],
 [36.54746072662782],
 [9.518167514307232],
 [7.148233024488038],
 [3.133496318698835],
 [6.933326075784346],
 [1.4515550631871843],
 [0.21842116512391033]]

CH4

In [1208]:
#find snr of ch4
snr_ch4_total=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
for a in range(16):
    snr_ch4_total[a].append(np.sum((dn[a][indch4]/dnr[a][indch4])**2)**(1/2))

In [1209]:
snr_ch4_total

[[0.9556289615619233],
 [2.991525170004608],
 [9.91031329225611],
 [6.2110887632073934],
 [2.495340924390988],
 [10.109723558161923],
 [3.322299130189707],
 [1.7301602731485162],
 [5.589734373387681],
 [23.83455375227605],
 [6.8460549541401035],
 [4.755319240713385],
 [2.2290397630007983],
 [4.515684128371087],
 [1.1804770836978082],
 [0.17763349255619754]]

CO2

In [1210]:
#find snr of co2
snr_co2_total=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
for a in range(16):
    snr_co2_total[a].append(np.sum((dn[a][indco2]/dnr[a][indco2])**2)**(1/2))

In [1211]:
snr_co2_total

[[1.9198602641281393],
 [6.010669004277063],
 [21.589485980849737],
 [13.337070757023936],
 [5.295077541621202],
 [22.324549919962244],
 [7.1463194458102075],
 [3.682076886313123],
 [11.237043059884858],
 [50.05842554960608],
 [13.59654026419785],
 [9.831364040730127],
 [4.450947303511594],
 [9.458810950978174],
 [2.2376663223242876],
 [0.3366809849075566]]

In [1212]:
#pts[['Planet', 'Teq','SNR_o2','SNR_ch4','SNR_h2o','SNR_co2','SNR_total']]

In [1213]:
#find the snr of atmosphere total
Total_modern = (np.array(snr_ch4_total)**2+np.array(snr_co2_total)**2+np.array(snr_h2o_total)**2+np.array(snr_o2_total)**2)**(1/2)

In [1214]:
Total_modern

array([[ 2.61598215],
       [ 8.18966838],
       [30.02553814],
       [18.51458188],
       [ 7.33215722],
       [31.13403112],
       [ 9.9215587 ],
       [ 5.07654292],
       [15.30731176],
       [69.6688714 ],
       [18.41528993],
       [13.51887911],
       [ 6.02445843],
       [13.05753271],
       [ 2.97051064],
       [ 0.44696368]])

## list the results

In [1219]:
pts["SNR_o2"]=np.array(snr_o2_total)
pts["SNR_ch4"]=np.array(snr_ch4_total)
pts["SNR_h2o"]=np.array(snr_h2o_total)
pts["SNR_co2"]=np.array(snr_co2_total)
pts["SNR_total"]=np.array(Total_modern)
#pts["NO_OVER"]=np.array(snr_total)
pts["mag"]=mag_pt


pts[['Planet','SNR_total']]

,Planet,SNR_total
0,GJ 1002 c,2.615982
1,GJ 1061 d,8.189668
2,GJ 15 A b,30.025538
3,GJ 251 b,18.514582
4,GJ 273 b,7.332157
5,GJ 411 b,31.134031
6,GJ 625 b,9.921559
7,GJ 667 C c,5.076543
8,GJ 682 b,15.307312
9,GJ 887 b,69.668871


In [1220]:
SNR_table  = pts[['Planet','SNR_total','SNR_h2o','SNR_ch4','SNR_co2','SNR_o2']].sort_values(by='SNR_total',ascending=False).round(2)
#pts[['Planet','SNR_total']]
SNR_table.to_latex()

/var/folders/6t/cpjtlkn10wv603vnph9nr3sh0000gn/T/ipykernel_22589/1026330911.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  SNR_table.to_latex()


"\\begin{tabular}{llrrrrr}\n\\toprule\n{} &              Planet &  SNR\\_total &  SNR\\_h2o &  SNR\\_ch4 &  SNR\\_co2 &  SNR\\_o2 \\\\\n\\midrule\n9  &            GJ 887 b &      69.67 &    36.55 &    23.83 &    50.06 &   21.07 \\\\\n5  &            GJ 411 b &      31.13 &    16.61 &    10.11 &    22.32 &    9.63 \\\\\n2  &           GJ 15 A b &      30.03 &    16.01 &     9.91 &    21.59 &    8.99 \\\\\n3  &            GJ 251 b &      18.51 &     9.87 &     6.21 &    13.34 &    5.38 \\\\\n10 &       Proxima Cen b &      18.42 &     9.52 &     6.85 &    13.60 &    4.10 \\\\\n8  &            GJ 682 b &      15.31 &     7.99 &     5.59 &    11.24 &    3.60 \\\\\n11 &          Ross 128 b &      13.52 &     7.15 &     4.76 &     9.83 &    3.52 \\\\\n13 &         Wolf 1061 c &      13.06 &     6.93 &     4.52 &     9.46 &    3.55 \\\\\n6  &            GJ 625 b &       9.92 &     5.29 &     3.32 &     7.15 &    2.88 \\\\\n1  &           GJ 1061 d &       8.19 &     4.28 &     2.99 &     6.01

In [1221]:
SNR_table

,Planet,SNR_total,SNR_h2o,SNR_ch4,SNR_co2,SNR_o2
9,GJ 887 b,69.67,36.55,23.83,50.06,21.07
5,GJ 411 b,31.13,16.61,10.11,22.32,9.63
2,GJ 15 A b,30.03,16.01,9.91,21.59,8.99
3,GJ 251 b,18.51,9.87,6.21,13.34,5.38
10,Proxima Cen b,18.42,9.52,6.85,13.60,4.10
8,GJ 682 b,15.31,7.99,5.59,11.24,3.60
11,Ross 128 b,13.52,7.15,4.76,9.83,3.52
13,Wolf 1061 c,13.06,6.93,4.52,9.46,3.55
6,GJ 625 b,9.92,5.29,3.32,7.15,2.88
1,GJ 1061 d,8.19,4.28,2.99,6.01,1.93


# GJ 887 b

run 1 first

# check contrast form 10-3 to 10-6

In [59]:
n_9_3=((((pn_9e_elt+2* 10**(-3) *pn_9_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.001)**2 * pn_9e_elt**2)**(1/2)
n_9_4=((((pn_9e_elt+2* 10**(-4) *pn_9_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.001)**2 * pn_9e_elt**2)**(1/2)
n_9_6=((((pn_9e_elt+2* 10**(-6) *pn_9_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.001)**2 * pn_9e_elt**2)**(1/2)
n_9_5=((((pn_9e_elt+2* 10**(-5) *pn_9_elt)*trans_use+2*pnsky_1_elt)**(1/2) / nor_trans)**2 + (0.001)**2 * pn_9e_elt**2)**(1/2)

In [ ]:
print("SNR(h2o) with instrument contrast 10-3:",np.sum((dn[9][indh2o]/n_9_3[indh2o])**2)**(1/2))
print("SNR(h2o) with instrument contrast 10-4:",np.sum((dn[9][indh2o]/n_9_4[indh2o])**2)**(1/2))
print("SNR(h2o) with instrument contrast 10-5:",np.sum((dn[9][indh2o]/n_9_5[indh2o])**2)**(1/2))
print("SNR(h2o) with instrument contrast 10-6:",np.sum((dn[9][indh2o]/n_9_6[indh2o])**2)**(1/2))

In [ ]:
print("SNR(ch4) with instrument contrast 10-3:",np.sum((dn[9][indch4]/n_9_3[indch4])**2)**(1/2))
print("SNR(ch4) with instrument contrast 10-4:",np.sum((dn[9][indch4]/n_9_4[indch4])**2)**(1/2))
print("SNR(ch4) with instrument contrast 10-5:",np.sum((dn[9][indch4]/n_9_5[indch4])**2)**(1/2))
print("SNR(ch4) with instrument contrast 10-6:",np.sum((dn[9][indch4]/n_9_6[indch4])**2)**(1/2))

In [ ]:
print("SNR(co2) with instrument contrast 10-3:",np.sum((dn[9][indco2]/n_9_3[indco2])**2)**(1/2))
print("SNR(co2) with instrument contrast 10-4:",np.sum((dn[9][indco2]/n_9_4[indco2])**2)**(1/2))
print("SNR(co2) with instrument contrast 10-5:",np.sum((dn[9][indco2]/n_9_5[indco2])**2)**(1/2))
print("SNR(co2) with instrument contrast 10-6:",np.sum((dn[9][indco2]/n_9_6[indco2])**2)**(1/2))

In [ ]:
print("SNR(o2) with instrument contrast 10-3:",np.sum((dn[9][indo2]/n_9_3[indo2])**2)**(1/2))
print("SNR(o2) with instrument contrast 10-4:",np.sum((dn[9][indo2]/n_9_4[indo2])**2)**(1/2))
print("SNR(o2) with instrument contrast 10-5:",np.sum((dn[9][indo2]/n_9_5[indo2])**2)**(1/2))
print("SNR(o2) with instrument contrast 10-6:",np.sum((dn[9][indo2]/n_9_6[indo2])**2)**(1/2))

In [ ]:
print("SNR(total) with instrument contrast 10-3:",np.sum((dn[9][indtotno]/n_9_3[indtotno])**2)**(1/2))
print("SNR(total) with instrument contrast 10-4:",np.sum((dn[9][indtotno]/n_9_4[indtotno])**2)**(1/2))
print("SNR(total) with instrument contrast 10-5:",np.sum((dn[9][indtotno]/n_9_5[indtotno])**2)**(1/2))
print("SNR(total) with instrument contrast 10-6:",np.sum((dn[9][indtotno]/n_9_6[indtotno])**2)**(1/2))